# Quickstart: Agentic retrieval in Azure AI Search

Use this notebook to get started with [agentic retrieval](https://learn.microsoft.com/azure/search/search-agentic-retrieval-concept) in Azure AI Search, which integrates an Azure OpenAI chat completion model to process queries, retrieve relevant content from indexed documents, and generate natural-language answers.

Steps in this notebook include:

1. Creating and loading an `earth-at-night` search index.

1. Creating an `earth-knowledge-source` that targets your index.

1. Creating an `earth-knowledge-agent` that targets your knowledge source and an LLM for query planning and answer synthesis.

1. Using the agent to fetch, rank, and synthesize relevant information from the index.

This notebook provides a high-level demonstration of agentic retrieval. For more detailed guidance, see [Quickstart: Run agentic retrieval in Azure AI Search](https://learn.microsoft.com/azure/search/search-get-started-agentic-retrieval?pivots=programming-language-csharp).

## Prerequisites

+ An [Azure AI Search service](https://learn.microsoft.com/azure/search/search-create-service-portal) on the Basic tier or higher with [semantic ranker enabled](https://learn.microsoft.com/azure/search/semantic-how-to-enable-disable).

+ An [Azure AI Foundry project](https://learn.microsoft.com/azure/ai-foundry/how-to/create-projects) and Azure AI Foundry resource. When you create a project, the resource is automatically created.

+ A [supported chat completion model](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-create#supported-models). This sample uses `gpt-5`.

+ A text embedding model. This sample uses `text-embedding-3-large`.

+ [Visual Studio Code](https://code.visualstudio.com/download).

## Configure access

This notebook assumes that you're using Microsoft Entra ID for authentication and role assignments for authorization.

To configure role-based access:

1. Sign in to the [Azure portal](https://portal.azure.com).

1. On your Azure AI Search service:

   1. [Enable role-based access](https://learn.microsoft.com/azure/search/search-security-enable-roles).

   1. [Create a system-assigned managed identity](https://learn.microsoft.com/azure/search/search-howto-managed-identities-data-sources#create-a-system-managed-identity) on your Azure AI Search service.

   1. [Assign the following roles](https://learn.microsoft.com/azure/search/search-security-rbac#how-to-assign-roles-in-the-azure-portal) to yourself.

      + **Search Service Contributor**

      + **Search Index Data Contributor**

      + **Search Index Data Reader**

1. On your Azure AI Foundry resource, assign **Cognitive Services User** to the managed identity of your search service.

## Set up connections

The `sample.env` file contains environment variables for connections to Azure AI Search and Azure OpenAI in Azure AI Foundry. Agentic retrieval requires these connections for document retrieval, query planning, and query execution.

To set up the connections:

1. Sign in to the [Azure portal](https://portal.azure.com).

2. Get the endpoints for Azure AI Search (`https://your-search-service.search.windows.net`) and Azure OpenAI in Azure AI Foundry (`https://your-foundry-resource.openai.azure.com`).

3. Save the `sample.env` file as `.env` on your local system.

4. Update the `.env` file with the retrieved endpoints.

## Install packages and load connections

This step installs the packages for this notebook and establishes connections to Azure AI Search and Azure OpenAI in Azure AI Foundry.

In [1]:
#r "nuget: Azure.Search.Documents, 11.7.0-beta.7"
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget:dotenv.net, 4.0.0"

Installed Packages Azure.Identity, 1.15.0 Azure.Search.Documents, 11.7.0-beta.7 dotenv.net, 4.0.0

In [17]:
using dotenv.net;
using Azure.Identity;

// .env should be in the same directory as this notebook
DotEnv.Load(options: new DotEnvOptions(envFilePaths: new[] { ".env" }, ignoreExceptions: false));

// Get environment variables with defaults where appropriate
string searchEndpoint = Environment.GetEnvironmentVariable("SEARCH_ENDPOINT")
    ?? throw new InvalidOperationException("SEARCH_ENDPOINT isn't set.");
string aoaiEndpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT")
    ?? throw new InvalidOperationException("AOAI_ENDPOINT isn't set.");

string aoaiEmbeddingModel = "text-embedding-3-large";
string aoaiEmbeddingDeployment = "text-embedding-3-large";
string aoaiGptModel = "gpt-5";
string aoaiGptDeployment = "gpt-5";

string indexName = "earth-at-night";
string knowledgeSourceName = "earth-knowledge-source";
string knowledgeAgentName = "earth-knowledge-agent";

var credential = new DefaultAzureCredential();

## Create an index in Azure AI Search

This step creates a search index that contains plain text and vector content. You can use an existing index, but it must meet the criteria for [agentic retrieval workloads](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-index). The primary schema requirement is a semantic configuration with a `DefaultConfigurationName`.

In [32]:
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Indexes.Models;

// Define fields for the index
var fields = new List<SearchField>
{
    new SimpleField("id", SearchFieldDataType.String) { IsKey = true, IsFilterable = true, IsSortable = true, IsFacetable = true },
    new SearchField("page_chunk", SearchFieldDataType.String) { IsFilterable = false, IsSortable = false, IsFacetable = false },
    new SearchField("page_embedding_text_3_large", SearchFieldDataType.Collection(SearchFieldDataType.Single)) { VectorSearchDimensions = 3072, VectorSearchProfileName = "hnsw_text_3_large" },
    new SimpleField("page_number", SearchFieldDataType.Int32) { IsFilterable = true, IsSortable = true, IsFacetable = true }
};

// Define a vectorizer
var vectorizer = new AzureOpenAIVectorizer(vectorizerName: "azure_openai_text_3_large")
{
    Parameters = new AzureOpenAIVectorizerParameters
    {
        ResourceUri = new Uri(aoaiEndpoint),
        DeploymentName = aoaiEmbeddingDeployment,
        ModelName = aoaiEmbeddingModel
    }
};

// Define a vector search profile and algorithm
var vectorSearch = new VectorSearch()
{
    Profiles =
    {
        new VectorSearchProfile(
            name: "hnsw_text_3_large",
            algorithmConfigurationName: "alg"
        )
        {
            VectorizerName = "azure_openai_text_3_large"
        }
    },
    Algorithms =
    {
        new HnswAlgorithmConfiguration(name: "alg")
    },
    Vectorizers =
    {
        vectorizer
    }
};

// Define a semantic configuration
var semanticConfig = new SemanticConfiguration(
    name: "semantic_config",
    prioritizedFields: new SemanticPrioritizedFields
    {
        ContentFields = { new SemanticField("page_chunk") }
    }
);

var semanticSearch = new SemanticSearch()
{
    DefaultConfigurationName = "semantic_config",
    Configurations =
    {
        semanticConfig
    }
};

// Create the index
var index = new SearchIndex(indexName)
{
    Fields = fields,
    VectorSearch = vectorSearch,
    SemanticSearch = semanticSearch
};

// Create the index client and create or update the index
var indexClient = new SearchIndexClient(new Uri(searchEndpoint), credential);
await indexClient.CreateOrUpdateIndexAsync(index);

Console.WriteLine($"Index '{indexName}' created or updated successfully.");

Index 'earth-at-night' created or updated successfully.


## Upload sample documents

This notebook uses data from NASA's Earth at Night e-book. The data is retrieved from the [azure-search-sample-data](https://github.com/Azure-Samples/azure-search-sample-data) repository on GitHub and passed to the search client for indexing.

In [33]:
using System.Net.Http;
using System.Text.Json;
using Azure.Search.Documents;
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Models;

// Upload sample documents from the GitHub URL
string url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json";
var httpClient = new HttpClient();
var response = await httpClient.GetAsync(url);
response.EnsureSuccessStatusCode();
var json = await response.Content.ReadAsStringAsync();

var documents = JsonSerializer.Deserialize<List<Dictionary<string, object>>>(json);
var searchClient = new SearchClient(new Uri(searchEndpoint), indexName, credential);
var searchIndexingBufferedSender = new SearchIndexingBufferedSender<Dictionary<string, object>>(
    searchClient,
    new SearchIndexingBufferedSenderOptions<Dictionary<string, object>>
    {
        KeyFieldAccessor = doc => doc["id"].ToString(),
    }
);

await searchIndexingBufferedSender.UploadDocumentsAsync(documents);
await searchIndexingBufferedSender.FlushAsync();

Console.WriteLine($"Documents uploaded to index '{indexName}' successfully.");

Documents uploaded to index 'earth-at-night' successfully.


## Create a knowledge source

This step creates a knowledge source that targets the index you previously created. In the next step, you create a knowledge agent that uses the knowledge source to orchestrate agentic retrieval.

In [34]:
using Azure.Search.Documents.Indexes.Models;

// Create the knowledge source
var indexKnowledgeSource = new SearchIndexKnowledgeSource(
    name: knowledgeSourceName,
    searchIndexParameters: new SearchIndexKnowledgeSourceParameters(searchIndexName: indexName)
    {
        SourceDataSelect = "id,page_chunk,page_number"
    }
);

await indexClient.CreateOrUpdateKnowledgeSourceAsync(indexKnowledgeSource);
Console.WriteLine($"Knowledge source '{knowledgeSourceName}' created or updated successfully.");

Knowledge source 'earth-knowledge-source' created or updated successfully.


## Create a knowledge agent


This step creates a knowledge agent, which acts as a wrapper for your knowledge source and LLM deployment.

`ExtractiveData` is the default modality and returns content from your knowledge sources without generative alteration. However, this quickstart uses the `AnswerSynthesis` modality for LLM-generated answers that cite the retrieved content.

In [35]:
using Azure.Search.Documents.Indexes.Models;

var openAiParameters = new AzureOpenAIVectorizerParameters
{
    ResourceUri = new Uri(aoaiEndpoint),
    DeploymentName = aoaiGptDeployment,
    ModelName = aoaiGptModel
};

var agentModel = new KnowledgeAgentAzureOpenAIModel(azureOpenAIParameters: openAiParameters);

var outputConfig = new KnowledgeAgentOutputConfiguration
{
    Modality = KnowledgeAgentOutputConfigurationModality.AnswerSynthesis,
    IncludeActivity = true
};

// Create the knowledge agent
var agent = new KnowledgeAgent(
    name: knowledgeAgentName,
    models: new[] { agentModel },
    knowledgeSources: new KnowledgeSourceReference[] { new KnowledgeSourceReference(knowledgeSourceName) { IncludeReferences = true, IncludeReferenceSourceData = true, RerankerThreshold = (float?)2.5 }}
)
{
    OutputConfiguration = outputConfig
};
await indexClient.CreateOrUpdateKnowledgeAgentAsync(agent);
Console.WriteLine($"Knowledge agent '{knowledgeAgentName}' created or updated successfully.");

Knowledge agent 'earth-knowledge-agent' created or updated successfully.


## Set up messages

Messages are the input for the retrieval route and contain the conversation history. Each message includes a `role` that indicates its origin, such as `system` or `user`, and `content` in natural language. The LLM you use determines which roles are valid.

In [36]:
string instructions = @"
A Q&A agent that can answer questions about the Earth at night.
If you don't have the answer, respond with ""I don't know"".
";

var messages = new List<Dictionary<string, string>>
{
    new Dictionary<string, string>
    {
        { "role", "system" },
        { "content", instructions }
    }
};

## Use agentic retrieval to fetch results

This step runs the agentic retrieval pipeline to produce a grounded, citation-backed answer. Given the conversation history and retrieval parameters, your knowledge agent:

1. Analyzes the entire conversation to infer the user's information need.

1. Decomposes the compound query into focused subqueries.

1. Runs the subqueries concurrently against your knowledge source.

1. Uses semantic ranker to rerank and filter the results.

1. Synthesizes the top results into a natural-language answer.

In [39]:
using Azure.Search.Documents.Agents;
using Azure.Search.Documents.Agents.Models;

var agentClient = new KnowledgeAgentRetrievalClient(
    endpoint: new Uri(searchEndpoint),
    agentName: knowledgeAgentName,
    tokenCredential: new DefaultAzureCredential()
);

messages.Add(new Dictionary<string, string>
{
    { "role", "user" },
    { "content", @"
Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
" }
});

var retrievalResult = await agentClient.RetrieveAsync(
    retrievalRequest: new KnowledgeAgentRetrievalRequest(
        messages: messages
            .Where(message => message["role"] != "system")
            .Select(
            message => new KnowledgeAgentMessage(content: new[] { new KnowledgeAgentMessageTextContent(message["content"]) })  { Role = message["role"] }
            )
            .ToList()
    )
);

messages.Add(new Dictionary<string, string>
{
    { "role", "assistant" },
    { "content", (retrievalResult.Value.Response[0].Content[0] as KnowledgeAgentMessageTextContent).Text }
});

### Review the retrieval response, activity, and results

Because your knowledge agent is configured for answer synthesis, the retrieval response contains the following values:

+ `Response`: An LLM-generated answer to the query that cites the retrieved documents.

+ `Activity`: Detailed planning and execution information, including subqueries, reranking decisions, and intermediate steps.

+ `References`: Source documents and chunks that contributed to the answer.

In [41]:
(retrievalResult.Value.Response[0].Content[0] as KnowledgeAgentMessageTextContent).Text 

Suburban belts display larger December brightening than urban cores, despite higher absolute light levels in downtown areas, likely due to the expansive holiday light displays in residential neighborhoods and commercial areas that tend to be more prevalent in suburban settings. This contributes to a greater relative increase in lighting during the holiday season compared to urban cores, which might have higher baseline lighting that does not change as dramatically [ref_id:1][ref_id:3]. 

Additionally, the Phoenix nighttime street grid is sharply visible from space due to its organized layout and extensive street lighting that outlines the city's grid pattern, making it easier to discern from low-Earth orbit. In contrast, large stretches of the interstate between midwestern cities remain comparatively dim because these highways often traverse less densely populated or rural areas with fewer lighting installations. The more pronounced urban lighting, as seen in cities like Phoenix, contr

In [23]:
Console.WriteLine("Activity:");
foreach (var activity in retrievalResult.Value.Activity)
{
    Console.WriteLine($"Activity Type: {activity.GetType().Name}");
    string json = JsonSerializer.Serialize(
        activity,
        activity.GetType(),
        new JsonSerializerOptions { WriteIndented = true }
    );
    Console.WriteLine(json);
}

Console.WriteLine("Results");
foreach (var reference in retrievalResult.Value.References)
{
    Console.WriteLine($"Reference Type: {reference.GetType().Name}");
    string json = JsonSerializer.Serialize(
        reference,
        reference.GetType(),
        new JsonSerializerOptions { WriteIndented = true }
    );
    Console.WriteLine(json);
}

Activity:
Activity Type: KnowledgeAgentModelQueryPlanningActivityRecord
{
  "InputTokens": 2278,
  "OutputTokens": 153,
  "Id": 0,
  "ElapsedMs": 3420
}
Activity Type: KnowledgeAgentSearchIndexActivityRecord
{
  "SearchIndexArguments": {
    "Search": "Why do suburban belts show larger December brightening than urban cores despite higher absolute light levels downtown?",
    "Filter": null
  },
  "KnowledgeSourceName": "earth-knowledge-source",
  "QueryTime": "2025-09-23T13:44:16.903+00:00",
  "Count": 0,
  "Id": 1,
  "ElapsedMs": 901
}
Activity Type: KnowledgeAgentSearchIndexActivityRecord
{
  "SearchIndexArguments": {
    "Search": "Why is the Phoenix nighttime street grid sharply visible from space compared to dimmer stretches of interstate between Midwestern cities?",
    "Filter": null
  },
  "KnowledgeSourceName": "earth-knowledge-source",
  "QueryTime": "2025-09-23T13:44:17.33+00:00",
  "Count": 0,
  "Id": 2,
  "ElapsedMs": 427
}
Activity Type: KnowledgeAgentSemanticRerankerActi

## Continue the conversation

This step continues the conversation with your knowledge agent, building upon the previous messages and queries to retrieve relevant information from your knowledge source.

In [24]:
messages.Add(new Dictionary<string, string>
{
    { "role", "user" },
    { "content", "How do I find lava at night?" }
});


var retrievalResult = await agentClient.RetrieveAsync(
    retrievalRequest: new KnowledgeAgentRetrievalRequest(
            messages: messages
                .Where(message => message["role"] != "system")
                .Select(
                    message => new KnowledgeAgentMessage(content: new[] { new KnowledgeAgentMessageTextContent(message["content"]) })  { Role = message["role"] }
                )
                .ToList()
            )
    );

messages.Add(new Dictionary<string, string>
{
    { "role", "assistant" },
    { "content", (retrievalResult.Value.Response[0].Content[0] as KnowledgeAgentMessageTextContent).Text }
});


### Review the new retrieval response, activity, and results

In [25]:
(retrievalResult.Value.Response[0].Content[0] as KnowledgeAgentMessageTextContent).Text 

Lava can be found at night by using satellite imagery that captures thermal and infrared wavelengths, which highlight the hot lava as bright white or red areas against the cooler surroundings. For example, Landsat 8 imagery combines thermal, shortwave infrared, and near-infrared wavelengths to detect very hot lava flows, cooling lava, and areas obscured by clouds, as seen in the monitoring of Kilauea's lava flows in Hawaii [ref_id:0]. Similarly, the Operational Land Imager (OLI) and Thermal Infrared Sensor (TIRS) on Landsat 8 can highlight active vents and lava flows, as demonstrated during Mount Etna's flank eruption in Italy [ref_id:1]. Additionally, the VIIRS Day/Night Band (DNB) on the Suomi NPP satellite can capture nighttime images of glowing lava flows, using faint light sources such as moonlight to illuminate the scene, enabling the detection of lava activity at night, as shown in images of Mount Etna [ref_id:2][ref_id:5]. Thus, by utilizing satellite instruments that detect th

In [26]:
Console.WriteLine("Activity:");
foreach (var activity in retrievalResult.Value.Activity)
{
    Console.WriteLine($"Activity Type: {activity.GetType().Name}");
    string json = JsonSerializer.Serialize(
        activity,
        activity.GetType(),
        new JsonSerializerOptions { WriteIndented = true }
    );
    Console.WriteLine(json);
}

Console.WriteLine("Results");
foreach (var reference in retrievalResult.Value.References)
{
    Console.WriteLine($"Reference Type: {reference.GetType().Name}");
    string json = JsonSerializer.Serialize(
        reference,
        reference.GetType(),
        new JsonSerializerOptions { WriteIndented = true }
    );
    Console.WriteLine(json);
}

Activity:
Activity Type: KnowledgeAgentModelQueryPlanningActivityRecord
{
  "InputTokens": 2311,
  "OutputTokens": 81,
  "Id": 0,
  "ElapsedMs": 3944
}
Activity Type: KnowledgeAgentSearchIndexActivityRecord
{
  "SearchIndexArguments": {
    "Search": "How to locate lava flows at night",
    "Filter": null
  },
  "KnowledgeSourceName": "earth-knowledge-source",
  "QueryTime": "2025-09-23T13:44:34.427+00:00",
  "Count": 6,
  "Id": 1,
  "ElapsedMs": 405
}
Activity Type: KnowledgeAgentSearchIndexActivityRecord
{
  "SearchIndexArguments": {
    "Search": "Methods for detecting lava at night",
    "Filter": null
  },
  "KnowledgeSourceName": "earth-knowledge-source",
  "QueryTime": "2025-09-23T13:44:34.797+00:00",
  "Count": 2,
  "Id": 2,
  "ElapsedMs": 369
}
Activity Type: KnowledgeAgentSearchIndexActivityRecord
{
  "SearchIndexArguments": {
    "Search": "Safety tips for finding lava at night",
    "Filter": null
  },
  "KnowledgeSourceName": "earth-knowledge-source",
  "QueryTime": "2025-

## Clean up objects and resources

If you no longer need Azure AI Search or Azure AI Foundry, delete the resources from your Azure subscription. You can also start over by deleting individual objects.

### Delete the knowledge agent

In [27]:
await indexClient.DeleteKnowledgeAgentAsync(knowledgeAgentName);
System.Console.WriteLine($"Knowledge agent '{knowledgeAgentName}' deleted successfully.");

Knowledge agent 'earth-knowledge-agent' deleted successfully.


### Delete the knowledge source

In [ ]:
await indexClient.DeleteKnowledgeSourceAsync(knowledgeSourceName);
System.Console.WriteLine($"Knowledge source '{knowledgeSourceName}' deleted successfully.");

Knowledge source 'earth-knowledge-source' deleted successfully.


### Delete the search index

In [29]:
await indexClient.DeleteIndexAsync(indexName);
System.Console.WriteLine($"Index '{indexName}' deleted successfully.");

Index 'earth-at-night' deleted successfully.
